In [ ]:
!pip install transformers --quiet

In [6]:
!type python3.9

python3.9 is /media/rkozik/02FF-A831/repos/swarog_exp/env.hator/bin/python3.9


In [1]:
!id="18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV"; conf=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$id -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'); wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$conf&id=$id" -O pubhealth.csv && rm -rf /tmp/cookies.txt

--2022-08-25 12:33:21--  https://docs.google.com/uc?export=download&confirm=&id=18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV
Resolving docs.google.com (docs.google.com)... 142.250.203.206, 2a00:1450:401b:810::200e
Connecting to docs.google.com (docs.google.com)|142.250.203.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hu3hf9fb264l5mne6gmhhtkla9f5dc4e/1661423550000/05536367513713019519/*/18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV?e=download&uuid=a2cba134-b25d-4da1-a1bb-a764bceac670 [following]
--2022-08-25 12:33:24--  https://doc-0k-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hu3hf9fb264l5mne6gmhhtkla9f5dc4e/1661423550000/05536367513713019519/*/18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV?e=download&uuid=a2cba134-b25d-4da1-a1bb-a764bceac670
Resolving doc-0k-34-docs.googleusercontent.com (doc-0k-34-docs.googleusercontent.com)... 142.250.75.1, 2a00:1450:40

In [1]:
import pandas as pd
data = pd.read_csv("../../raw/politifact.csv",sep=",")
data.head()

,Unnamed: 0,speaker,text,explanation,url,label
0,0,Battleground Texas,"Says Dan Patrick has ""called immigration into ...",'Stop the illlegal invasion!',/texas/statements/2014/jun/20/battleground-tex...,0
1,1,Battleground Texas,"In 2008, ""only 54 percent of Latinos in Texas ...",Dividing estimates into estimates,/texas/statements/2013/mar/22/battleground-tex...,0
2,2,Suzanne Somers,"Even after Obamacare is fully implemented, the...","Some ridiculed her column, but this claim is c...",/punditfact/statements/2013/nov/01/suzanne-som...,0
3,3,Jamie Oliver,McDonald's in England only sells organic milk ...,Policies differ under Golden Arches across the...,/truth-o-meter/statements/2010/aug/31/jamie-ol...,0
4,4,John Barge,We have about six school districts that are in...,Tight budgets tightening school days,/georgia/statements/2013/oct/29/john-barge/bar...,0


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

data = pd.read_csv("../../raw/politifact.csv",sep=",")
print("labels\n",data["label"].value_counts())
print("shape  \n",data.shape)

bootstrap_size = 0

if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / data.shape[0]
    bootstrap = np.random.uniform(size=data.shape[0]) < bootstrap_factor
    data = data.iloc[bootstrap]

X, y = data["text"].values, data["label"].values

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

labels
 0    7639
1    2824
Name: label, dtype: int64
shape  
 (10463, 6)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1878.33it/s]

shapes X (10463,) y (10463,)


In [3]:
data.columns

Index(['Unnamed: 0', 'speaker', 'text', 'explanation', 'url', 'label'], dtype='object')

In [3]:
data["label"].value_counts()

0    7639
1    2824
Name: label, dtype: int64

In [1]:
import transformers

In [4]:
import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast

from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np

from scipy.sparse import csr_matrix
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device:", device)

if "disilbert_model" not in locals():
    disilbert_tokenizer =  AutoTokenizer.from_pretrained("distilbert-base-uncased")
    disilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")
    handle = disilbert_model.to(device)


class BERTEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer =  disilbert_tokenizer
        self.model = disilbert_tokenizer
        self.max_length = 256
        self.model_name = disilbert_model

    def fit(self, X=None, y=None):
        pass
    
    def encode(self, txt):
        return self.tokenizer(txt, max_length=self.max_length, 
                              truncation=True, padding=True, return_tensors="pt")

    def transform(self, X=None):
        dataloader = DataLoader(X, batch_size=16, shuffle=False)
        allembeds = []
        for batch in tqdm(dataloader):
            batchenc = disilbert_tokenizer(batch, max_length=256, truncation=True, padding=True, return_tensors="pt")
            input_ids = batchenc['input_ids'].to(device)
            attention_mask = batchenc['attention_mask'].to(device)
            batchout = disilbert_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            embeds = [vec[0].cpu().detach().numpy() for vec in batchout[1][-1]]
            allembeds.extend(embeds)
        return csr_matrix(allembeds)


class BertHead(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.head = LogisticRegression(class_weight='auto', max_iter=10000)

    def fit(self, X=None, y=None):
        self.head.fit(X, y)


    def transform(self, X=None):
        pass
    
    def predict(self, X=None):    
        return self.head.predict(X)



using device: cuda


In [5]:
bert = BERTEmbeddings()
X_dstil_numpy = bert.transform(X).toarray()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 654/654 [00:04<00:00, 130.98it/s]


In [6]:
import pickle

# SAVE
with open('politifact_BERTEmbeddings.pickle', 'wb') as handle:
    pickle.dump(X_dstil_numpy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
import pickle

# LOAD
with open('politifact_BERTEmbeddings.pickle', 'rb') as handle:
    X_dstil_numpy = pickle.load(handle)

In [7]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.linear_model import LogisticRegression

arr = []

for fold,j in enumerate(foldids):
    train = foldids[fold][1]
    test = foldids[fold][2]
    xin, yin = X_dstil_numpy[train], np.array(y[train])
    cls = BertHead()
    
    cls.fit(xin, yin)
    y_pred = cls.predict(X_dstil_numpy[test])

    bac = balanced_accuracy_score(y[test], y_pred)
    arr.append(bac)

    print(" BAC=", bac)

print(10*"-")
print("AVG. BAC=",np.mean(arr),"+/-",np.std(arr))

 BAC= 0.5512221348797888
 BAC= 0.5457295860046717
 BAC= 0.5461337377452798
 BAC= 0.5535524999128407
 BAC= 0.5504215920383252
 BAC= 0.5484254031764542
 BAC= 0.5541321210862762
 BAC= 0.5509489417383042
 BAC= 0.5570257923853877
 BAC= 0.5499164754726442
----------
AVG. BAC= 0.5507508284439973 +/- 0.0033415804699697405


In [8]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.model_selection import RepeatedStratifiedKFold


def experiment(foldids, X, y, cls = LogisticRegression(max_iter=10000), fit=True):

    scores = {
        'Accuracy': {'func': accuracy_score},
        'Balanced Accuracy': {'func': balanced_accuracy_score},
        'F1': {'func': f1_score},
        'Precision': {'func': precision_score},
        'Recall': {'func': recall_score},
        'G-mean': {'func': geometric_mean_score}
    }

    for score_name, score_dict in scores.items():
        scores[score_name]["list"] = []
        scores[score_name]["lab"] = []

    for fold,j in enumerate(foldids):
        train = foldids[fold][1]
        test = foldids[fold][2]
        xin, yin = X[train], np.array(y[train])
        
        pca = PCA(n_components=512)
        pca.fit(xin)
        
        
        if fit == True:
            cls.fit(pca.transform(xin), yin)
        y_pred = cls.predict(pca.transform(X[test]))
        for score_name, score_dict in scores.items():
            if score_name in ["F1","Precision","Recall"]:
                scorvaln = score_dict['func'](y[test], y_pred, average=None)
                score_dict['lab'].append(scorvaln)
                scorval = score_dict['func'](y[test], y_pred, average="weighted")
                score_dict['list'].append(scorval)
                #print(score_name, scorval, scorvaln)  
            else:
                scorval=score_dict['func'](y[test], y_pred)
                score_dict['list'].append(scorval)
                #print(score_name, scorval)
        #print(" ")

    #clear_output()
    for score_name, score_dict in scores.items():
        score_dict['avg'] = np.mean(score_dict['list'])
        score_dict['std'] = np.std(score_dict['list'])
 
    # Print stats
    numlabels = scores["F1"]["lab"][0].shape[0]
    scores["F1"]["lab"][0].shape[0] 
    head = "| %-20s | %-10s |" +  numlabels * " %-10s |" 
    headv = ["Score", "Average"]
    headv.extend(["Kat_"+str(i+1) for i in range(numlabels)])
    row=head % tuple(headv)
    print("+"*len(row))
    print(row)
    print("+"*len(row))
    for score_name, score_dict in sorted(scores.items()) :
        headv = [score_name, np.mean(score_dict['list'])*100, np.std(score_dict['list'])*100]
        for i in range(numlabels):
            if score_name in ["F1","Precision","Recall"]:
                head = "| %-20s | %4.1f ± %3.1f |" + numlabels* " %4.1f ± %3.1f |"
                vals = [v[i] for v in scores[score_name]["lab"]]
                headv.append(np.mean(vals)*100)
                headv.append(np.std(vals)*100)
            else:
                head = "| %-20s | %4.1f ± %3.1f |" + numlabels * " %-10s |" 
                headv.append("-")
        print(head % tuple(headv))
    print("+"*len(row))
    return cls, scores, pca

In [9]:
foldids = []
for fold_idx, (train, test) in enumerate(rskf.split(X, y)):
    foldids.append((fold_idx,train,test))

#print("shapes X",X.shape,"y", y.shape)
model1, scores1, pca = experiment(foldids, X_dstil_numpy, y, 
                             LogisticRegression(max_iter=10000))
    


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Score                | Average    | Kat_1      | Kat_2      |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Accuracy             | 71.9 ± 0.2 | -          | -          |
| Balanced Accuracy    | 54.9 ± 0.3 | -          | -          |
| F1                   | 67.2 ± 0.2 | 82.7 ± 0.2 | 25.5 ± 0.8 |
| G-mean               | 40.4 ± 0.8 | -          | -          |
| Precision            | 67.0 ± 0.3 | 75.2 ± 0.1 | 44.9 ± 0.8 |
| Recall               | 71.9 ± 0.2 | 91.9 ± 0.4 | 17.8 ± 0.7 |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
